<a href="https://colab.research.google.com/github/young-hyun-park/capston_design/blob/main/densenet121_multilabelclassfication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [1]:
import torch
import numpy as np
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchsummary import summary
from PIL import Image
import os
import re
import nibabel as nib

## Data Load

In [2]:
zip_path = sorted(glob('/content/drive/Shareddrives/캡스톤 디자인1/dataset/original_512/*'))

In [3]:
import zipfile
for path in zip_path:
  zip_file = zipfile.ZipFile(path)
  zip_file.extractall('/content/datasets/'+path.split('/')[-1].split('.')[0])
  zip_file.close()

In [4]:
PATH = '/content/datasets/'
from glob import glob
file_data = glob(PATH+'*')

In [5]:
my_glob = glob('/content/datasets/images*/*.png')
print('Number of Observations: ', len(my_glob))


Number of Observations:  76666


## Data Load

In [6]:
import pandas as pd
df= pd.read_csv('/content/drive/Shareddrives/캡스톤 디자인1/processed_data_entry.csv')

In [7]:
df.drop(['Follow-up #', 'Patient ID',
       'Patient Age', 'Patient Gender', 'View Position', 'OriginalImage[Width','Height]', 'OriginalImagePixelSpacing[x', 'y]'],axis = 1,inplace = True)

In [8]:
full_img_paths = {os.path.basename(x): x for x in my_glob}
df['full_path'] = df['Image Index'].map(full_img_paths.get)

In [9]:
classes = ['Atelectasis',
                'Consolidation',
                'Infiltration', 
                'Pneumothorax', 
                'Edema', 
                'Emphysema',
                'Fibrosis', 
                'Effusion',
                'Pneumonia',
                'Pleural_Thickening',
                'Cardiomegaly',
                'Nodule', 
                'Mass', 
                'Hernia']
for label in classes:
  df[label] = df['Finding Labels'].map(lambda result: 1.0 if label in result else 0)

In [10]:
data = df[classes]
df['labels'] = data.apply(lambda row: np.argmax(row) if np.sum(row)>0 else -1, axis = 1)

In [11]:
df = df[df['Finding Labels']!= 'No Finding']
df.reset_index(drop = True, inplace = True)

In [12]:
df.columns

Index(['Image Index', 'Finding Labels', 'full_path', 'Atelectasis',
       'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema', 'Emphysema',
       'Fibrosis', 'Effusion', 'Pneumonia', 'Pleural_Thickening',
       'Cardiomegaly', 'Nodule', 'Mass', 'Hernia', 'labels'],
      dtype='object')

In [13]:
target_cols = df.drop(['Image Index','Finding Labels', 'full_path','labels'], axis=1).columns.to_list()

In [14]:
target_cols

['Atelectasis',
 'Consolidation',
 'Infiltration',
 'Pneumothorax',
 'Edema',
 'Emphysema',
 'Fibrosis',
 'Effusion',
 'Pneumonia',
 'Pleural_Thickening',
 'Cardiomegaly',
 'Nodule',
 'Mass',
 'Hernia']

In [15]:
from sklearn.model_selection import train_test_split
paths = df['full_path'].values
columns = df.columns[3:-1]
labels = df[columns].values
(train_path, val_path ,train_labels , val_labels) = train_test_split(paths,labels,test_size = 0.2,random_state = 42)

In [16]:
val_labels[1]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0.])

In [17]:
class Train_Dataset(Dataset):
    def __init__(self, data_path,labels, transform = None):
        self.data_path = data_path
        self.transform = transform
        self.labels = labels
    def __len__(self):
        return len(self.data_path)
    def __getitem__(self,idx):
        path = self.data_path[idx]
        img = np.array(Image.open(path))
        img = img/255
        img= np.stack((img,)*3, axis=-1)
        label = self.labels[idx]
        if self.transform is not None:
            transformed = self.transform(image=img)
            image = transformed['image']
        return image, label

In [18]:
class Val_Dataset(Dataset):
    def __init__(self, data_path,labels, transform = None):
        self.data_path = data_path
        self.transform = transform
        self.labels = labels
    def __len__(self):
        return len(self.data_path)
    def __getitem__(self,idx):
        path = self.data_path[idx]
        img = np.array(Image.open(path))
        img = img/255
        img= np.stack((img,)*3, axis=-1)
        label = self.labels[idx]
        if self.transform is not None:
            transformed = self.transform(image=img)
            image = transformed['image']
        return image, label

In [19]:
#!pip install --upgrade --force-reinstall --no-deps albumentations
!pip install albumentations==0.4.6

     |████████████████████████████████| 117 kB 14.3 MB/s 
     |████████████████████████████████| 948 kB 59.1 MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65174 sha256=5e6a1515013c8575aee100d8f9be7815eada747e8425f1d7d3ca835cadcd691b
  Stored in directory: /root/.cache/pip/wheels/cf/34/0f/cb2a5f93561a181a4bcc84847ad6aaceea8b5a3127469616cc
Successfully built albumentations
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [20]:
import albumentations as A                                                                           
from albumentations.pytorch import ToTensorV2

In [21]:
data_transforms = {
    'train': A.Compose([
     ToTensorV2()
     ]
    ),
    'val': A.Compose([
                      ToTensorV2()   
    ]
                           )
}
'''
A.RandomCrop(224, 224),
     A.OneOf([
              A.HorizontalFlip(p=1),
              A.RandomRotate90(p=1),
              A.VerticalFlip(p=1)            
    ], p=1), 
'''

'\nA.RandomCrop(224, 224),\n     A.OneOf([\n              A.HorizontalFlip(p=1),\n              A.RandomRotate90(p=1),\n              A.VerticalFlip(p=1)            \n    ], p=1), \n'

In [22]:
# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]

# Number of classes in the dataset

num_classes = 14

# Batch size for training (change depending on how much memory you have)
batch_size = 16
patience = 7
# Number of epochs to train for
num_epochs = 5

In [23]:
train_data = Train_Dataset(train_path,train_labels,transform = data_transforms['train'])
val_data = Val_Dataset(val_path,val_labels,transform =  data_transforms['val'])

In [24]:
image_datasets = {'train' : train_data , 'val' : val_data}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True) for x in ['train', 'val']}
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Model

In [25]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [26]:
pip install timm

     |████████████████████████████████| 431 kB 15.2 MB/s 


In [27]:
import timm

In [28]:
#timm.list_models(pretrained = True)

In [29]:
model= torch.load('/content/drive/Shareddrives/캡스톤 디자인1/codes/densenet121_multilabelclassfication_1_10epoch.pt')

In [30]:
#model.classifier = nn.Linear(in_features=1024, out_features=14, bias=True)

In [31]:
device = torch.device('cuda:0')

In [32]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score , roc_curve , accuracy_score,classification_report,multilabel_confusion_matrix
import tqdm

In [33]:
import torch.optim as optim
from torch.optim import lr_scheduler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ft = model.to(device)

criterion = nn.BCEWithLogitsLoss()

# specify optimizer
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.0001)
sgdr_partial = lr_scheduler.CosineAnnealingLR(optimizer_ft, T_max=5, eta_min=0.005 )

In [34]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='/content/drive/Shareddrives/캡스톤 디자인1/codes/resnet34_num14_dropout_augmented_multilabelclassfication_checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            #self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            #self.save_checkpoint(val_loss, model)
            self.counter = 0

In [35]:
def train_model(model, dataloaders, criterion, optimizer, scheduler , num_epochs=25):
    since = time.time()
    train_acc_history = []
    train_loss_hist = [] 
    train_classification_report =[]
    train_confusion_matrix = []
    val_loss_hist = []
    val_acc_history = [] 
    val_classification_report =[]
    val_confusion_matrix = []
    epoch_train_label = []
    epoch_train_output = []
    epoch_val_label = []
    epoch_val_output = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_loss = 1000000000
    best_train_loss = 100000000
    early_stopping = EarlyStopping(patience = patience, verbose = True)
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            start = time.time()
            print('phase : %r' %phase)
            if phase == 'train':
                model.train() # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_acc = 0
            running_precision =0
            running_recall = 0
            running_f1_score =0 
            running_auc = 0
            # Iterate over data.
            full_label = []
            full_output = []
            full_prediction = []
            for inputs, labels in tqdm.notebook.tqdm(dataloaders[phase]):
                inputs = inputs.to(device = device, dtype = torch.float32)
                labels = labels.to(device = device)
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    outputs = model(inputs)
                    loss = criterion(outputs,labels.float())
                    pred = nn.Sigmoid()(outputs)
                    full_prediction += pred.detach().cpu().numpy().tolist()
                    pred[pred <0.5] = 0
                    pred[pred >=0.5] = 1
                    if phase == 'train':
                    # backward + optimize only if in training phase
                        loss.backward()
                        optimizer.step()
                        scheduler.step()
                # statistics
                running_loss += loss.item() * inputs.size(0)
                full_label+= labels.detach().cpu().numpy().tolist()
                full_output+= pred.detach().cpu().numpy().tolist()
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = accuracy_score(full_label,full_output)
            epoch_classification_report = classification_report(full_label,full_output,target_names = target_cols)
            epoch_multilabel_confusion_matrix = multilabel_confusion_matrix(full_label,full_output)
             
            if phase == 'train':
              train_acc_history.append(epoch_acc)
              train_loss_hist.append(epoch_loss)
              epoch_train_label.append(full_label)
              epoch_train_output.append(full_prediction)
              train_classification_report.append(epoch_classification_report)
              train_confusion_matrix.append(epoch_multilabel_confusion_matrix)
            print('\n{} Loss: {:.4f} Acc: {:.4f}, Time : {:.4f}'.format(phase, epoch_loss ,epoch_acc,time.time()-start))
            #,   , , epoch_precision,epoch_recall, epoch_auc, epoch_f1_score 
            print('\n--------------------------------')
            print('|{}Epoch {}phase multilabel_confusion_matrix report|'.format(epoch+1,phase))
            print('--------------------------------')
            print('\n{}'.format(epoch_multilabel_confusion_matrix))
            print('\n--------------------------------')
            print('\n{}Epoch {}phase classification report'.format(epoch+1,phase))
            print('--------------------------------')
            print(epoch_classification_report)

            # deep copy the model
            if phase == 'val' and train_loss_hist[-1] < best_train_loss:
                best_val_loss = epoch_loss
                best_train_loss = train_loss_hist[-1]
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
                val_loss_hist.append(epoch_loss)
                epoch_val_label.append(full_label)
                epoch_val_output.append(full_prediction)
                val_classification_report.append(epoch_classification_report)
                val_confusion_matrix.append(multilabel_confusion_matrix)
                early_stopping(epoch_loss, model)


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best train loss : {:.4f}  Best val loss: {:4f}'.format(best_train_loss ,best_val_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    train_dict = {'Loss' : train_loss_hist , 'Accuarcy' : train_acc_history,'Classification_report' : train_classification_report,'Confusion_matrix' :train_confusion_matrix, 'train_label' : epoch_train_label , 'train_output' : epoch_train_output}
    #
    val_dict = {'Loss' :val_loss_hist , 'Accuarcy' : val_acc_history,'Classification_report' : val_classification_report,'val_label' : epoch_val_label , 'Confusion_matrix' :val_confusion_matrix,'val_output' : epoch_val_output}
    #, , 
    return model, train_dict ,val_dict

In [36]:
#summary(model.to('cuda'),(3,512,512))

In [37]:
criterion = nn.BCEWithLogitsLoss()
import pickle
# Train and evaluate
for i in range(5):
  start = i*10+11
  end = (i+1)*10+10
  print('\n--------------------------------')
  print('|        Epoch {} ~ {}      |'.format(start, end))
  print('\n--------------------------------')
  model_ft, train_dict, val_dict  = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft,sgdr_partial, num_epochs=10)
  torch.save(model_ft, '/content/drive/Shareddrives/캡스톤 디자인1/codes/densenet121_multilabelclassfication_{}_{}epoch.pt'.format(start, end))
  with open('/content/drive/Shareddrives/캡스톤 디자인1/model_history/densenet121_multilabelclassfication_{}_{}epoch_train_dict.pkl'.format(start, end),'wb') as fw:
    pickle.dump(train_dict, fw)
  with open('/content/drive/Shareddrives/캡스톤 디자인1/model_history/densenet121_multilabelclassfication_{}_{}epoch_val_dict.pkl'.format(start, end),'wb') as ff:
    pickle.dump(val_dict, ff)


'''

confusion maxrix
[[true negative , false positive
false negative , true positive]]

[[tn , fp
fn , tp]]
'''


--------------------------------
|        Epoch 11 ~ 20      |

--------------------------------
Epoch 1/10
----------
phase : 'train'


  0%|          | 0/2588 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



train Loss: 0.2380 Acc: 0.2034, Time : 1130.3562

--------------------------------
|1Epoch trainphase multilabel_confusion_matrix report|
--------------------------------

[[[30632  1545]
  [ 6514  2716]]

 [[37651     4]
  [ 3746     6]]

 [[21413  4119]
  [ 8983  6892]]

 [[36499   624]
  [ 2912  1372]]

 [[39507    78]
  [ 1721   101]]

 [[39165   270]
  [ 1380   592]]

 [[40048     5]
  [ 1351     3]]

 [[28549  2214]
  [ 5169  5475]]

 [[40268     0]
  [ 1139     0]]

 [[38648    36]
  [ 2686    37]]

 [[38848   341]
  [ 1410   808]]

 [[36073   267]
  [ 4722   345]]

 [[36241   527]
  [ 3243  1396]]

 [[41216     9]
  [  136    46]]]

--------------------------------

1Epoch trainphase classification report
--------------------------------
                    precision    recall  f1-score   support

       Atelectasis       0.64      0.29      0.40      9230
     Consolidation       0.60      0.00      0.00      3752
      Infiltration       0.63      0.43      0.51     15875
  

  0%|          | 0/647 [00:00<?, ?it/s]

...


val Loss: 0.7157 Acc: 0.1704, Time : 168.5417

--------------------------------
|7Epoch valphase multilabel_confusion_matrix report|
--------------------------------

[[[ 7253   770]
  [ 1588   741]]

 [[ 8980   457]
  [  786   129]]

 [[ 4060  2273]
  [ 1761  2258]]

 [[ 8923   411]
  [  684   334]]

 [[ 9680   191]
  [  401    80]]

 [[ 9731    77]
  [  399   145]]

 [[ 9956    64]
  [  322    10]]

 [[ 6948   731]
  [ 1504  1169]]

 [[10014    46]
  [  291     1]]

 [[ 9545   145]
  [  625    37]]

 [[ 9570   224]
  [  355   203]]

 [[ 8422   666]
  [  998   266]]

 [[ 8579   630]
  [  706   437]]

 [[10300     7]
  [   38     7]]]

--------------------------------

7Epoch valphase classification report
--------------------------------
                    precision    recall  f1-score   support

       Atelectasis       0.49      0.32      0.39      2329
     Consolidation       0.22      0.14      0.17       915
      Infiltration       0.50      0.56      0.53      4019
      Pne

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


  0%|          | 0/2588 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'PngImageFile'.  With the exception of `AttributeError` NumPy will always raise this exception in the future.  Raise this deprecation warning to see the original exception. (Warning added NumPy 1.21)
  # Remove the CWD from sys.path while we load stuff.


TypeError: ignored